# 위키문헌 작품목록 텍스트 스크래핑
- 작성자: [지해인](https://haein.info)
- 개별 문서 불러오기 이전까지는 기존 코드를 사용했습니다.

## 환경 설정

In [1]:
# 필요한 라이브러리 설치
!pip install mwxml requests tqdm pandas

# 기본 라이브러리 임포트
import mwxml
import bz2
import json
import os
import re
import urllib.parse
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd
import requests

print("모든 라이브러리가 성공적으로 로드되었습니다!")
print(f"사용 가능한 CPU 코어: {mp.cpu_count()}개")

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d39387f9f423a50d7b5a1ed05f3b9a832d6beeb794c9d9ea371fd70ef6e25504
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
모든 라이브러리가 성공적으로 로드되었습니다!
사용 가능한 CPU 코어: 2개


## 데이터 다운로드

In [2]:
# 위키문헌 덤프 다운로드
dump_url = "https://dumps.wikimedia.org/kowikisource/20251001/kowikisource-20251001-pages-articles.xml.bz2"
dump_file = "kowikisource-20251001-pages-articles.xml.bz2"

if not os.path.exists(dump_file):
    print(" 위키문헌 덤프 다운로드 중... (약 150MB, 시간이 걸릴 수 있습니다)")
    !wget -O {dump_file} {dump_url}
    print("다운로드 완료!")
else:
    print("덤프 파일이 이미 존재합니다!")

# 파일 크기 확인
file_size = os.path.getsize(dump_file) / (1024 * 1024)  # MB
print(f"파일 크기: {file_size:.1f} MB")

 위키문헌 덤프 다운로드 중... (약 150MB, 시간이 걸릴 수 있습니다)
--2025-10-22 03:43:09--  https://dumps.wikimedia.org/kowikisource/20251001/kowikisource-20251001-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155556940 (148M) [application/octet-stream]
Saving to: ‘kowikisource-20251001-pages-articles.xml.bz2’

kowikisource-202510 100%[===================>] 148.35M  3.79MB/s    in 43s     

2025-10-22 03:43:53 (3.46 MB/s) - ‘kowikisource-20251001-pages-articles.xml.bz2’ saved [155556940/155556940]

다운로드 완료!
파일 크기: 148.4 MB


## API 보강 함수

In [3]:
def get_categories_from_api(page_title):
    """
    위키문헌 API에서 페이지의 모든 분류를 가져옵니다

    이 함수가 중요한 이유:
    - XML 덤프에는 기본 분류만 있음
    - 템플릿에서 자동 생성되는 분류는 API로만 확인 가능
    - 예: '1941년 작품', 'PD-old-50' 등
    """
    try:
        api_url = "https://ko.wikisource.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'categories',
            'cllimit': 'max'
        }

        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        data = response.json()

        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1':
            return []

        categories = pages[page_id].get('categories', [])
        category_names = []

        for cat in categories:
            cat_title = cat['title']
            if cat_title.startswith('분류:'):
                category_names.append(cat_title[3:])  # '분류:' 제거

        return category_names

    except Exception as e:
        print(f"API 분류 조회 오류 ({page_title}): {e}")
        return []

def get_year_from_wikidata(page_title):
    """
    위키데이터에서 작품의 발표 연도를 가져옵니다

    위키데이터 연동의 장점:
    - 구조화된 연도 정보 제공
    - 여러 언어판에서 공유되는 정확한 데이터
    - 분류 정보와 교차 검증 가능
    """
    try:
        # 1단계: 위키문헌 페이지에서 위키데이터 ID 가져오기
        wikisource_api = "https://ko.wikisource.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'pageprops'
        }

        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(wikisource_api, params=params, headers=headers, timeout=10)
        data = response.json()

        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1':
            return None

        wikidata_id = pages[page_id].get('pageprops', {}).get('wikibase_item')

        if not wikidata_id:
            return None

        # 2단계: 위키데이터에서 발표일 정보 가져오기
        wikidata_api = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbgetentities',
            'format': 'json',
            'ids': wikidata_id,
            'props': 'claims'
        }

        response = requests.get(wikidata_api, params=params, headers=headers, timeout=10)
        data = response.json()

        entity = data.get('entities', {}).get(wikidata_id, {})
        claims = entity.get('claims', {})

        # 발표 관련 속성들 확인
        date_properties = ['P577', 'P571', 'P585']  # 발표일, 시작일, 특정시점

        for prop in date_properties:
            if prop in claims:
                for claim in claims[prop]:
                    try:
                        time_value = claim['mainsnak']['datavalue']['value']['time']
                        # +1941-00-00T00:00:00Z 형태에서 연도 추출
                        year = int(time_value[1:5])
                        if 1800 <= year <= 2030:  # 유효한 연도 범위
                            return year
                    except (KeyError, ValueError):
                        continue

        return None

    except Exception as e:
        print(f"위키데이터 조회 오류 ({page_title}): {e}")
        return None

def enhance_with_api(page_data):
    """
    페이지 데이터를 API 정보로 보강합니다

    보강 과정:
    1. API에서 완전한 분류 정보 가져오기
    2. 위키데이터에서 연도 정보 가져오기
    3. 분류에서 연도 추출하기
    4. 최종 연도 결정 (분류 > 위키데이터 > 덤프)
    """
    title = page_data['title']

    # API에서 완전한 분류 정보 가져오기
    api_categories = get_categories_from_api(title)

    # 위키데이터에서 연도 정보 가져오기
    wikidata_year = get_year_from_wikidata(title)

    # 기존 데이터 복사
    enhanced = page_data.copy()

    # 분류 정보 병합 (중복 제거)
    all_categories = list(set(page_data.get('categories', []) + api_categories))
    enhanced['categories'] = all_categories
    enhanced['api_categories'] = api_categories

    # 분류에서 연도 추출
    year_from_categories = None
    year_categories = [cat for cat in all_categories if '년' in cat and '작품' in cat]
    if year_categories:
        for cat in year_categories:
            year_match = re.search(r'(\d{4})년', cat)
            if year_match:
                year_from_categories = int(year_match.group(1))
                break

    # 최종 연도 결정 (우선순위: 분류 > 위키데이터 > 덤프)
    enhanced['year'] = (
        year_from_categories or
        wikidata_year or
        page_data.get('year')
    )

    # 보강 정보 추가
    enhanced['year_from_categories'] = year_from_categories
    enhanced['year_from_wikidata'] = wikidata_year

    return enhanced

print("API 보강 함수들이 정의되었습니다!")

API 보강 함수들이 정의되었습니다!


## 텍스트 파싱 함수

In [4]:
def extract_metadata(text):
    """
    위키텍스트에서 메타데이터를 추출합니다

    추출하는 정보:
    - 저자: [[저자:이름]] 패턴에서
    - 분류: [[분류:이름]] 패턴에서
    - 작곡가: '작곡' 키워드 주변에서
    - 라이선스: {{PD-*}} 템플릿에서
    - 언어: '한자', '한글' 키워드에서
    """
    if not text:
        return {
            'authors': [],
            'categories': [],
            'composer': None,
            'translator': None,
            'year': None,
            'license': None,
            'language': None
        }

    # 저자 정보 추출
    authors = []
    author_patterns = [
        r'\[\[저자:([^|\]]+)',  # [[저자:이름]] 또는 [[저자:이름|표시명]]
        r'\|\s*author\s*=\s*\[\[저자:([^|\]]+)',  # author= 매개변수
    ]

    for pattern in author_patterns:
        matches = re.findall(pattern, text)
        authors.extend(matches)

    # 분류 정보 추출
    categories = re.findall(r'\[\[분류:([^\]]+)\]\]', text)

    # 작곡가 정보 추출
    composer = None
    composer_patterns = [
        r'([^.]+)\s*작곡',
        r'작곡가?\s*[:=]\s*([^.\n]+)'
    ]
    for pattern in composer_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            composer = matches[0].strip()
            break

    # 연도 정보 추출
    year_matches = re.findall(r'(\d{4})년', text)
    year = None
    if year_matches:
        year = max(set(year_matches), key=year_matches.count)

    # 라이선스 정보 추출
    license_info = None
    license_patterns = [
        r'\{\{(PD-[^}]+)\}\}',
        r'\{\{(CC-[^}]+)\}\}'
    ]
    for pattern in license_patterns:
        matches = re.findall(pattern, text)
        if matches:
            license_info = matches[0]
            break

    # 언어 정보 추출
    language = None
    if '한자' in text and '한글' in text:
        language = '한자+한글'
    elif '한자' in text:
        language = '한자'
    elif '한글' in text:
        language = '한글'

    # 중복 제거
    authors = list(set([a.strip() for a in authors if a.strip()]))
    categories = list(set([c.strip() for c in categories if c.strip()]))

    return {
        'authors': authors,
        'categories': categories,
        'composer': composer,
        'translator': None,  # 나중에 확장 가능
        'year': year,
        'license': license_info,
        'language': language
    }

def clean_text(text):
    """
    위키텍스트에서 마크업을 제거하고 깔끔한 본문만 추출합니다

    제거하는 것들:
    - 템플릿: {{...}}
    - HTML 태그: <div>, <br> 등
    - 위키 링크: [[...]]
    - 마크업: ''', ''
    """
    if not text:
        return ""

    # 템플릿 제거
    cleaned = re.sub(r'\{\{[^{}]*\}\}', '', text)

    # 링크에서 텍스트만 추출
    cleaned = re.sub(r'\[\[[^|\]]*\|([^\]]+)\]\]', r'\1', cleaned)
    cleaned = re.sub(r'\[\[([^\]]+)\]\]', r'\1', cleaned)

    # HTML 태그 제거
    cleaned = re.sub(r'<[^>]+>', '', cleaned)

    # 위키 마크업 제거
    cleaned = re.sub(r"'''([^']+)'''", r'\1', cleaned)  # 굵은 글씨
    cleaned = re.sub(r"''([^']+)''", r'\1', cleaned)   # 기울임

    # 섹션 헤더 제거
    cleaned = re.sub(r'^=+\s*([^=]+)\s*=+$', r'\1', cleaned, flags=re.MULTILINE)

    # 여러 공백을 하나로
    cleaned = re.sub(r'\s+', ' ', cleaned)

    return cleaned.strip()

def generate_urls(title, authors):
    """
    페이지와 저자의 URL을 생성합니다
    """
    base_url = "https://ko.wikisource.org/wiki/"

    # 페이지 URL
    page_url = base_url + urllib.parse.quote(title.replace(' ', '_'))

    # 저자 URL들
    author_links = []
    for author in authors:
        author_url = base_url + urllib.parse.quote(f"저자:{author}".replace(' ', '_'))
        author_links.append({
            'name': author,
            'url': author_url
        })

    return page_url, author_links

print("텍스트 파싱 함수들이 정의되었습니다!")

텍스트 파싱 함수들이 정의되었습니다!


## 메인 파서

In [5]:
def process_pages_batch(pages_batch):
    """
    페이지 배치를 처리하는 워커 함수
    (멀티프로세싱에서 각 코어가 실행)
    """
    results = []

    for page_data in pages_batch:
        try:
            page_id, title, namespace, redirect, revisions = page_data

            if not revisions:
                continue

            # 최신 리비전 사용
            revision = revisions[0]
            revision_id, timestamp, username, comment, text, size = revision

            # 메타데이터 추출
            metadata = extract_metadata(text)

            # 본문 정리
            clean_content = clean_text(text)

            # URL 생성
            page_url, author_links = generate_urls(title, metadata['authors'])

            # 페이지 데이터 구성
            page_result = {
                'page_id': page_id,
                'title': title,
                'url': page_url,
                'namespace': namespace,
                'redirect': redirect,

                # 필수 메타데이터
                'authors': metadata['authors'],
                'author_links': author_links,
                'categories': metadata['categories'],
                'content': clean_content,
                'raw_content': text,

                # 추가 메타데이터
                'composer': metadata['composer'],
                'translator': metadata['translator'],
                'year': metadata['year'],
                'license': metadata['license'],
                'language': metadata['language'],

                # 리비전 정보
                'revision_id': revision_id,
                'last_modified': str(timestamp) if timestamp else None,
                'last_contributor': username,
                'size': len(text) if text else 0,
                'content_size': len(clean_content)
            }

            results.append(page_result)

        except Exception as e:
            print(f"페이지 처리 오류: {e}")
            continue

    return results

def parse_wikisource(dump_file, limit=None, enable_api=False, batch_size=50):
    """
    위키문헌 덤프를 파싱합니다

    Args:
        dump_file: 덤프 파일 경로
        limit: 처리할 페이지 수 제한 (None이면 전체)
        enable_api: API 보강 사용 여부
        batch_size: 배치 크기

    Returns:
        list: 파싱된 페이지 데이터
    """
    start_time = time.time()

    # CPU 코어 수 (Colab에서 최대 활용)
    max_workers = mp.cpu_count()

    print(f" 위키문헌 파싱 시작!")
    print(f"   CPU 코어: {max_workers}개 (최대 활용)")
    print(f"   배치 크기: {batch_size}")
    print(f"   API 보강: {'사용' if enable_api else '사용 안함'}")

    # 1단계: 덤프에서 데이터 수집
    print("\n 1단계: 덤프 데이터 수집")
    pages_batches = []
    current_batch = []
    total_pages = 0

    with bz2.open(dump_file, 'rt', encoding='utf-8') as f:
        dump = mwxml.Dump.from_file(f)

        for page in tqdm(dump, desc="페이지 수집"):
            if limit and total_pages >= limit:
                break

            try:
                # 리비전 데이터 수집
                revisions = []
                for revision in page:
                    revisions.append((
                        revision.id,
                        revision.timestamp,
                        revision.user.text if revision.user else None,
                        revision.comment,
                        revision.text,
                        revision.bytes
                    ))
                    break  # 최신 리비전만

                page_data = (
                    page.id,
                    page.title,
                    page.namespace,
                    str(page.redirect.title) if page.redirect else None,
                    revisions
                )
                current_batch.append(page_data)

                if len(current_batch) >= batch_size:
                    pages_batches.append(current_batch)
                    current_batch = []

                total_pages += 1

            except Exception as e:
                continue

        if current_batch:
            pages_batches.append(current_batch)

    print(f" {total_pages}개 페이지를 {len(pages_batches)}개 배치로 수집")

    # 2단계: 멀티프로세싱으로 병렬 처리
    print("\n 2단계: 멀티프로세싱 처리")
    results = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 모든 배치를 병렬로 처리
        futures = [executor.submit(process_pages_batch, batch) for batch in pages_batches]

        for future in tqdm(as_completed(futures), total=len(futures), desc="배치 처리"):
            try:
                batch_results = future.result()
                results.extend(batch_results)
            except Exception as e:
                print(f"배치 처리 오류: {e}")

    # 3단계: API 보강 (선택적)
    if enable_api and results:
        print("\n 3단계: API 보강 처리")
        enhanced_results = []

        for page_data in tqdm(results, desc="API 보강"):
            try:
                enhanced = enhance_with_api(page_data)
                enhanced_results.append(enhanced)
                time.sleep(0.1)  # API 제한 고려
            except Exception as e:
                print(f"API 보강 오류 ({page_data.get('title', 'Unknown')}): {e}")
                enhanced_results.append(page_data)

        results = enhanced_results

    total_time = time.time() - start_time
    print(f"\n파싱 완료!")
    print(f"  총 시간: {total_time:.1f}초")
    print(f"  처리 속도: {len(results)/total_time:.1f} 페이지/초")
    print(f"  총 페이지: {len(results)}개")

    return results

print("메인 파서가 정의되었습니다!")

메인 파서가 정의되었습니다!


## 대량 파싱 예시

In [ ]:
# 대량 처리 실행
print(" 실습 3: 대량 처리")
print("="*50)

# 처리할 페이지 수 설정 (필요에 따라 조정)
BULK_LIMIT = 50  # 50개 페이지 처리
API_ENHANCEMENT = True  # API 보강 사용 여부

print(f" 설정:")
print(f"  처리 페이지: {BULK_LIMIT}개")
print(f"  API 보강: {'사용' if API_ENHANCEMENT else '사용 안함'}")
print(f"  CPU 활용: {mp.cpu_count()}개 코어 최대 활용")

# 대량 처리 실행
bulk_results = parse_wikisource(
    dump_file,
    limit=BULK_LIMIT,
    enable_api=API_ENHANCEMENT,
    batch_size=10  # 배치 크기
)

print(f"\n 대량 처리 완료!")
print(f"처리된 페이지: {len(bulk_results)}개")

 실습 3: 대량 처리
 설정:
  처리 페이지: 50개
  API 보강: 사용
  CPU 활용: 2개 코어 최대 활용
 위키문헌 파싱 시작!
   CPU 코어: 2개 (최대 활용)
   배치 크기: 10
   API 보강: 사용

 1단계: 덤프 데이터 수집


페이지 수집: 50it [00:00, 220.99it/s]

 50개 페이지를 5개 배치로 수집

 2단계: 멀티프로세싱 처리



배치 처리: 100%|██████████| 5/5 [00:11<00:00,  2.39s/it]



 3단계: API 보강 처리


API 보강: 100%|██████████| 50/50 [00:20<00:00,  2.41it/s]


파싱 완료!
  총 시간: 33.0초
  처리 속도: 1.5 페이지/초
  총 페이지: 50개

 대량 처리 완료!
처리된 페이지: 50개


# 개별 문서 불러오기

## 각 함수의 역할 설명

### 1. `extract_page_title(link_url)`: 주소에서 제목만 추출

* **역할:** CSV 파일에 있는 긴 **위키문헌 웹 주소**(URL)에서 실제 API 요청에 사용할 수 있는 **깨끗한 작품 제목**만 분리해내는 '주소 정리 담당'입니다.
* **작동 방식:** URL에서 `https://ko.wikisource.org/wiki/` 부분을 제거하고, 제목 부분에 있는 URL 인코딩(%)을 원래 문자로 되돌리며, 공백을 나타내는 밑줄(`_`)을 일반 공백으로 바꿉니다.
* **예시:** `'https://ko.wikisource.org/wiki/혈의_누'` $\rightarrow$ **`'혈의 누'`**

### 2. `get_wikitext_from_api(page_title)`: 실시간 원본 텍스트 배달

* **역할:** 추출한 작품 제목을 가지고 위키문헌 서버에 접속하여, 해당 작품의 **최신 원본 위키텍스트**(Raw Content)를 가져오는 '데이터 수집 담당'입니다.
* **작동 방식:** 위키문헌 API에 요청을 보내 해당 페이지의 **가장 최근 수정본**의 내용(`text`)과 수정 시간, 수정한 사용자 등의 **리비전 정보**를 받아옵니다.
* **결과 사용:** 이 함수가 가져온 원본 텍스트는 다음 단계인 `extract_metadata`와 `clean_text` 함수의 입력값으로 사용됩니다.

### 3. `process_csv_links_with_api(csv_file)`: 전체 작업 흐름 관리 및 통합

* **역할:** CSV 파일 로드부터 최종 데이터 보강 및 정리까지 **전체 작업 과정을 총괄**하는 '총 책임자' 함수입니다.
* **작동 방식:**
    1.  **CSV 읽기:** 입력된 CSV 파일에서 '링크' 열의 주소들을 하나씩 읽습니다.
    2.  **순회 및 추출:** 각 링크에 대해 `extract_page_title`을 호출하여 제목을 얻고, `get_wikitext_from_api`를 호출하여 원본 텍스트를 가져옵니다.
    3.  **정리 및 구성:** 가져온 원본 텍스트를 이용해 `extract_metadata` (정보 추출)와 `clean_text` (본문 정리) 함수들을 차례로 호출하여 기본적인 데이터를 만듭니다.
    4.  **보강 (가장 중요):** `enhance_with_api` 함수를 호출하여 위키데이터 연도, 숨겨진 분류 등 **추가적이고 정확한 정보를 덧붙여(보강하여)** 최종 결과 목록(`final_data`)에 저장합니다.
    5.  **반환:** 모든 작품의 처리가 끝나면 정리된 최종 데이터 목록을 반환합니다.

In [31]:
def get_wikitext_from_api(page_title):
    """
    위키문헌 API를 사용하여 페이지의 위키텍스트(raw content)를 가져옴
    """
    try:
        api_url = "https://ko.wikisource.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'revisions',
            'rvprop': 'content|ids|timestamp|user|size|comment',
            'rvslots': 'main',
            'rvlimit': '1'
        }

        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        data = response.json()

        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1':
            return None, None # 텍스트 없음, 리비전 정보 없음

        revisions = pages[page_id].get('revisions', [])
        if not revisions:
            return None, None

        revision = revisions[0]
        # 'content' 또는 '*' 필드에서 텍스트를 가져옴
        text = revision.get('slots', {}).get('main', {}).get('*')

        revision_info = {
            'revision_id': revision.get('revid'),
            'last_modified': revision.get('timestamp'),
            'last_contributor': revision.get('user'),
            'size': revision.get('size'),
            'comment': revision.get('comment')
        }

        return text, revision_info

    except Exception as e:
        print(f"API 위키텍스트 조회 오류 ({page_title}): {e}")
        return None, None

def extract_page_title(link_url):
    """
    위키문헌 URL에서 페이지 제목을 추출하고 디코딩
    예: 'https://ko.wikisource.org/wiki/혈의_누' -> '혈의 누'
    """
    try:
        # URL 디코딩
        decoded_url = urllib.parse.unquote(link_url)
        # '/wiki/' 뒤의 부분을 가져와서 밑줄을 공백으로 변환
        title_with_underscores = decoded_url.split('/wiki/')[1]
        title = title_with_underscores.replace('_', ' ')
        return title
    except:
        return None

def process_csv_links_with_api(csv_file='한국근대소설_TEI_XML_작품목록.csv'):
    """
    CSV 파일에서 링크를 읽고 API를 통해 위키문헌 데이터를 추출 및 보강합니다.
    """
    print(f"\n CSV 파일 로드: {csv_file}")

    # 1. CSV 파일 로드
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f" 오류: '{csv_file}' 파일을 찾을 수 없습니다. 파일을 업로드했는지 확인해주세요.")
        return []

    if '링크' not in df.columns:
        print(" 오류: CSV 파일에 '링크' 열이 없습니다. 열 이름을 확인해주세요.")
        return []

    valid_links = df['링크'].dropna().astype(str)
    print(f" 총 {len(valid_links)}개의 링크 발견. 데이터 추출 시작...")

    final_data = []

    # tqdm을 사용하여 진행 상황 표시
    for index, link_url in tqdm(valid_links.items(), total=len(valid_links), desc="작품별 API 처리"):
        if not link_url.startswith('https://ko.wikisource.org/wiki/'):
            continue

        page_title = extract_page_title(link_url)

        if not page_title:
            continue

        # 2. 위키텍스트 및 기본 정보 API로 가져오기
        raw_content, revision_info = get_wikitext_from_api(page_title)

        if not raw_content:
            continue

        # 3. 기존 정의된 함수들로 메타데이터 추출 및 본문 정리
        metadata = extract_metadata(raw_content)
        clean_content = clean_text(raw_content)
        page_url, author_links = generate_urls(page_title, metadata['authors'])

        # 기본 데이터 구조 생성
        page_data = {
            'page_id': revision_info.get('revision_id', None),
            'title': page_title,
            'url': page_url,
            'namespace': 0,
            'redirect': None,

            # 메타데이터
            'authors': metadata['authors'],
            'author_links': author_links,
            'categories': metadata['categories'],
            'content': clean_content,
            'raw_content': raw_content,

            'composer': metadata['composer'],
            'translator': metadata['translator'],
            'year': metadata['year'], # 덤프 파싱으로 추출된 year (API 보강 후 최종 year 결정)
            'license': metadata['license'],
            'language': metadata['language'],

            # 리비전 정보
            'revision_id': revision_info.get('revision_id'),
            'last_modified': revision_info.get('last_modified'),
            'last_contributor': revision_info.get('last_contributor'),
            'size': len(raw_content) if raw_content else 0,
            'content_size': len(clean_content)
        }

        # 4. API 보강 (완전한 분류, 위키데이터 연도 등)
        enhanced_data = enhance_with_api(page_data)
        final_data.append(enhanced_data)

        # API 요청 제한을 위한 대기 (매우 중요)
        time.sleep(0.3)

    print(f"\n 총 {len(final_data)}개의 작품 데이터 추출 및 보강 완료.")

    return final_data

print("API 호출 및 제목 추출 등을 위한 함수 정의 완료.")

API 호출 및 제목 추출 등을 위한 함수 정의 완료.


## 1. `dataframe_to_xml(df)`: 데이터를 XML 문서 구조로 변환

* **쉽게 말하면:** 정리된 표 형태의 데이터(**DataFrame**)를 **XML**(eXtensible Markup Language)이라는 계층적 문서 형태로 바꿔주는 **'XML 제작 공장'** 입니다. Pandas의 기본 `to_xml()` 기능에 문제가 있을 때 사용되는 **대체 솔루션**입니다.

* **핵심 작동 방식:**
    1.  **준비:** 데이터를 담을 최상위 태그인 `<works>`를 만듭니다. (XML의 뿌리)
    2.  **행(Row) 순회:** DataFrame의 각 행(작품 하나)을 반복하면서, `<work>` 태그를 하나씩 만듭니다.
    3.  **태그 배치:**
        * `title`, `url`, `year` 같은 단순 값들은 `<title>...</title>` 같은 **일반 태그**로 배치합니다.
        * `authors`, `categories`처럼 여러 개가 있는 목록(List)은 `<authors>` 안에 `<author>...</author>`를 여러 개 만드는 **중첩 구조**로 만듭니다.
        * `page_id`, `revision_id` 같은 부가 정보는 `<work page_id="..." revision_id="...">`처럼 **속성**(Attribute)으로 추가합니다.
    4.  **최종 정리:** 만들어진 복잡한 XML 구조를 사람이 읽기 쉽도록 줄 바꿈과 들여쓰기(`Pretty Print`)를 적용한 후, 파일로 저장할 수 있는 형태로 반환합니다.

## 2. `output_data(df, ...)`: 최종 파일을 선택하고 저장

* **쉽게 말하면:** 변환된 데이터를 **사용자가 원하는 형식(JSON, CSV, TSV, XML)**으로 파일 저장 작업을 수행하는 **'출력 관리자'** 입니다.

* **핵심 작동 방식:**
    1.  **형식 결정:** 입력받은 `output_format` 인수에 따라 최종 파일 이름(`extracted_wikisource_modern_novels.xml` 등)을 결정합니다.
    2.  **분기 처리:**
        * **JSON/CSV/TSV:** Pandas DataFrame이 기본으로 제공하는 `to_json()`, `to_csv()` 함수를 사용하여 파일을 저장합니다.
        * **XML:** **`dataframe_to_xml`** 함수를 호출하여 XML 문자열을 받아온 후, 이를 파일에 직접 작성하는 방식으로 저장합니다.
    3.  **오류 처리:** 파일을 저장하는 과정에서 오류가 발생하면 사용자에게 알리고, 특히 불안정한 XML 저장이 실패할 경우 **CSV로 대체 저장**하도록 안전장치를 마련합니다.

In [32]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def dataframe_to_xml(df):
    """
    Pandas DataFrame을 지정된 포맷의 XML 문자열로 변환합니다.
    """
    root = ET.Element('works')

    # NaN 값을 빈 문자열로 처리하여 XML 변환 시 오류를 방지합니다.
    df = df.fillna('')

    for index, row in df.iterrows():
        work = ET.SubElement(root, 'work')

        # 'title', 'url', 'authors', 'year' 등 주요 필드를 태그로 추가
        for col in ['title', 'url', 'year', 'license']:
            elem = ET.SubElement(work, col)
            elem.text = str(row[col])

        # authors 목록 처리
        authors_elem = ET.SubElement(work, 'authors')
        if isinstance(row['authors'], list):
             for author_name in row['authors']:
                author_elem = ET.SubElement(authors_elem, 'author')
                author_elem.text = str(author_name)

        # categories 목록 처리
        categories_elem = ET.SubElement(work, 'categories')
        if isinstance(row['categories'], list):
            for cat_name in row['categories']:
                cat_elem = ET.SubElement(categories_elem, 'category')
                cat_elem.text = str(cat_name)

        # 'content' (깔끔한 본문) 처리
        content_elem = ET.SubElement(work, 'content')
        content_elem.text = row['content']

        # 주석 정보 (리비전 정보 등)는 속성으로 추가
        work.set('page_id', str(row['page_id']))
        work.set('revision_id', str(row['revision_id']))

    # Pretty Print (들여쓰기)를 적용하여 사람이 읽기 쉽게 만듭니다.
    xml_string = ET.tostring(root, encoding='utf-8')
    reparsed = minidom.parseString(xml_string)
    return reparsed.toprettyxml(indent="  ", encoding='utf-8')

print("XML 변환 헬퍼 함수 정의 완료.")

def output_data(df, base_filename='extracted_wikisource_modern_novels', output_format='json'):
    """
    DataFrame을 지정된 형식(json, csv, tsv, xml)으로 저장합니다.
    """
    output_format = output_format.lower()
    output_filename = f"{base_filename}.{output_format}"

    print(f"\n파일 형식: {output_format.upper()}로 저장 중...")

    try:
        if output_format == 'json':
            df.to_json(output_filename, orient='records', force_ascii=False, indent=4)
        elif output_format == 'csv':
            df.to_csv(output_filename, index=False, encoding='utf-8')
        elif output_format == 'tsv':
            df.to_csv(output_filename, index=False, sep='\t', encoding='utf-8')
        elif output_format == 'xml':
            # 수동 XML 변환 함수 사용
            xml_output = dataframe_to_xml(df)
            with open(output_filename, 'wb') as f:
                f.write(xml_output)
        else:
            print(f"경고: 지원하지 않는 형식 '{output_format}'. JSON으로 저장합니다.")
            output_filename = f"{base_filename}.json"
            df.to_json(output_filename, orient='records', force_ascii=False, indent=4)

        print(f" 데이터가 '{output_filename}'으로 저장되었습니다. 다운로드하여 확인하세요.")
    except Exception as e:
        print(f" 데이터 저장 오류 ({output_format}): {e}")
        # XML 저장이 실패하면, CSV로 대체 저장하는 옵션도 고려할 수 있습니다.
        if output_format == 'xml':
             print("대체: XML 저장에 실패하여 CSV로 저장합니다.")
             df.to_csv(f"{base_filename}_fallback.csv", index=False, encoding='utf-8')

XML 변환 헬퍼 함수 정의 완료.


In [33]:
# 3. 결과 확인 및 DataFrame 변환
extracted_works = process_csv_links_with_api()

# 결과를 DataFrame으로 변환하여 분석 및 저장
if extracted_works:
    results_df = pd.DataFrame(extracted_works)
    print("\n 추출된 데이터 요약 (상위 5개)")
    print(results_df[['title', 'authors', 'year', 'year_from_wikidata', 'year_from_categories', 'license', 'url', 'content_size']].head())

    # 형식 선택: 'json', 'csv', 'tsv', 'xml' 중 하나를 선택하세요.
    desired_format = 'xml'

    # output_data 함수 실행
    output_data(results_df, output_format=desired_format)


 CSV 파일 로드: 한국근대소설_TEI_XML_작품목록.csv
 총 42개의 링크 발견. 데이터 추출 시작...


작품별 API 처리: 100%|██████████| 42/42 [01:47<00:00,  2.56s/it]


 총 42개의 작품 데이터 추출 및 보강 완료.

 추출된 데이터 요약 (상위 5개)
  title      authors  year  year_from_wikidata  year_from_categories  \
0  혈의 누        [이인직]  1906              1906.0                1906.0   
1   철세계  [이해조, 쥘 베른]  None                 NaN                   NaN   
2   자유종        [이해조]  1910                 NaN                1910.0   
3  화의 혈        [이해조]  1911                 NaN                1911.0   
4   추월색        [최찬식]  1912                 NaN                1912.0   

           license                                                url  \
0       PD-old-100  https://ko.wikisource.org/wiki/%ED%98%88%EC%9D...   
1       PD-old-100  https://ko.wikisource.org/wiki/%EC%B2%A0%EC%84...   
2  PD-US|1927|1910  https://ko.wikisource.org/wiki/%EC%9E%90%EC%9C...   
3        PD-old-70  https://ko.wikisource.org/wiki/%ED%99%94%EC%9D...   
4       PD-old-100  https://ko.wikisource.org/wiki/%EC%B6%94%EC%9B...   

   content_size  
0           347  
1         58894  
2           303  
3      

# 덤프 파일 보강

### `find_related_dump_pages`: **대용량 위키문헌 덤프 파일($\text{kowikisource-...xml.bz2}$)에서 필요한 작품의 모든 텍스트 조각을 찾아 모으는** 역할
* 이 함수는 CSV에 있는 **메인 작품**뿐만 아니라, 그 작품의 **장(Chapter)이나 절(Section)이 분리되어 저장된 모든 하위 페이지**를 덤프 전체를 뒤져서 찾아냅니다.

In [34]:
import re
import bz2
from tqdm import tqdm
import mwxml # mwxml 라이브러리는 이미 임포트되어 있어야 합니다.

def find_related_dump_pages(dump_file, target_titles):
    """
    덤프 파일에서 목표 작품 제목과 관련된 모든 페이지(본문 및 하위 페이지/장)를 찾습니다.
    (하위 페이지 매칭을 최대로 강화)
    """
    related_pages = {}
    current_titles = set(target_titles)

    print(f"\n 덤프 파일에서 {len(current_titles)}개 작품의 관련 페이지 찾기 시작...")

    # 덤프에서 가져올 네임스페이스 (0: 본문, 102: 쪽)
    TARGET_NAMESPACES = {0, 102}

    try:
        with bz2.open(dump_file, 'rt', encoding='utf-8') as f:
            dump = mwxml.Dump.from_file(f)

            for page in tqdm(dump, desc="덤프 페이지 탐색 중"):
                if page.namespace not in TARGET_NAMESPACES or page.redirect:
                    continue

                text = next(page).text # 최신 리비전 텍스트 추출

                is_related = False

                for target_title in current_titles:
                    # -------------------------------------------------------------
                    # 1. Namespace 0 (본문 및 하위 페이지) 매칭
                    # -------------------------------------------------------------
                    if page.namespace == 0:
                        # 1) 정확히 일치 (메인 제목)
                        if page.title == target_title:
                            is_related = True
                            break

                        # 2) CSV 제목으로 시작하는 모든 하위 페이지 매칭 (가장 강력한 매칭)
                        # 예: '장한몽'과 '장한몽/1장'
                        if page.title.startswith(target_title + '/'):
                            is_related = True
                            break

                        # 3) 인코딩된 형식 매칭 (공백 대신 밑줄)
                        if page.title.startswith(target_title.replace(' ', '_') + '/'):
                            is_related = True
                            break

                    # -------------------------------------------------------------
                    # 2. Namespace 102 (쪽 문서) 매칭
                    # -------------------------------------------------------------
                    elif page.namespace == 102:
                        # 쪽 문서의 베이스 제목이 CSV 작품 제목과 관련 있는지 확인
                        match_base = re.search(r'^(?:[^/]+)', page.title.split(':', 1)[-1])
                        if match_base:
                            base_title = match_base.group(0).strip()
                            if target_title in base_title or base_title in target_title:
                                is_related = True
                                break

                if is_related:
                    if page.title not in related_pages:
                        related_pages[page.title] = {
                            'title': page.title,
                            'raw_content': text,
                            'namespace': page.namespace,
                            'revisions': [text]
                        }

    except Exception as e:
        print(f"덤프 파싱 중 치명적인 오류 발생: {e}")

    print(f" 덤프에서 총 {len(related_pages)}개의 관련 페이지(본문, 하위 장, 쪽 문서) 수집 완료.")
    return related_pages

### 1. `sort_wikisource_parts(page_content_list)`: 숫자 순서대로 장(Chapter) 정렬

* **역할:** 덤프에서 가져온 작품의 **분리된 텍스트 조각들**($\text{/1장, /2장, .../10장}$)을 **숫자($\text{1, 2, 10}$)를 인식**하여 사람이 읽는 순서대로 정확하게 **줄 세우는** 함수입니다.
* **핵심:** '10'이 '2'보다 먼저 오지 않도록 **논리적인 순서**를 보장하여 본문 조합 오류를 막습니다.

### 2. `process_csv_links_with_dump(csv_file, dump_file)`: 덤프 데이터 통합 및 최종 완성

* **역할:** CSV 목록을 기준으로 대용량 덤프($\text{XML}$)에서 필요한 데이터를 모두 **찾아 모으고, 합치고, 정리**하여 최종 결과물을 만드는 **'총괄 지휘 본부'** 입니다.
* **핵심 과정:**
    1.  **수집:** $\text{find\_related\_dump\_pages}$를 호출해 메인 페이지와 모든 **하위 장/쪽 문서**를 가져옵니다.
    2.  **조합:** $\text{sort\_wikisource\_parts}$로 순서를 맞춘 후, $\text{clean\_text}$로 마크업을 제거하며 **하나의 완성된 본문**으로 합칩니다.
    3.  **보강:** $\text{enhance\_with\_api}$로 최신 분류와 위키데이터 연도를 **추가 보강**하여 데이터셋을 완성합니다.

In [37]:
def sort_wikisource_parts(page_content_list):
    """
    위키문헌 쪽 문서나 하위 장 제목을 숫자 기반으로 정렬합니다.
    (예: .../10장 보다 .../2장이 뒤에 오지 않도록)
    """
    def extract_key(item):
        title = item['title']
        # 숫자와 알파벳을 포함하는 부분을 추출하고, 나머지는 문자열로 남깁니다.
        # 예: '장한몽/1장' -> ('장한몽/', 1, '장')
        # 예: '쪽:파일명.pdf/10' -> ('쪽:파일명.pdf/', 10, '')
        parts = re.split(r'(\d+)', title)

        # 숫자 부분은 정수로 변환, 나머지 문자열은 그대로 둠
        key = []
        for part in parts:
            if part.isdigit():
                key.append(int(part))
            elif part:
                key.append(part)

        return key

    return sorted(page_content_list, key=extract_key)


def process_csv_links_with_dump(csv_file='한국근대소설_TEI_XML_작품목록.csv', dump_file="kowikisource-20251001-pages-articles.xml.bz2"):
    # (CSV 로드 및 target_titles 추출 로직은 이전과 동일)
    try:
        df = pd.read_csv(csv_file)
        target_titles = {extract_page_title(url) for url in df['링크'].dropna() if url.startswith('https://ko.wikisource.org')}
        target_titles.discard(None)
    except Exception as e:
        print(f" CSV 처리 오류: {e}")
        return []
    if not target_titles:
        print(" 오류: 유효한 작품 제목을 추출할 수 없습니다.")
        return []

    # 1. 덤프 파일에서 모든 관련 페이지 데이터 수집
    dump_data = find_related_dump_pages(dump_file, target_titles)

    final_data = []

    for page_title in tqdm(target_titles, desc="작품별 데이터 조합 및 보강"):
        main_page = None

        # 2. 메인 작품 페이지 찾기
        for dump_key, data in dump_data.items():
            if data['namespace'] == 0 and data['title'] == page_title:
                main_page = data
                break

        if not main_page:
            continue

        raw_content = main_page['revisions'][0]
        metadata = extract_metadata(raw_content)
        page_url, author_links = generate_urls(page_title, metadata['authors'])

        # 3. 본문 내용 조합 (하위 페이지 + 쪽 문서)
        page_content_list = []

        # 작품 제목과 관련된 모든 하위/쪽 페이지 수집
        for dump_key, data in dump_data.items():
            # 메인 페이지는 제외
            if data['title'] == page_title:
                continue

            # 하위 페이지(Namespace 0) 또는 쪽 문서(Namespace 102)인지 확인
            is_subpage_or_page = False

            # 하위 페이지 형식: '제목/...' 또는 '제목_밑줄/...'
            if data['namespace'] == 0 and (data['title'].startswith(page_title + '/') or data['title'].startswith(page_title.replace(' ', '_') + '/')):
                is_subpage_or_page = True

            # 쪽 문서 형식: '쪽:파일명...'에서 파일명이 제목과 관련될 때
            elif data['namespace'] == 102:
                match_base = re.search(r'^(?:[^/]+)', data['title'].split(':', 1)[-1])
                if match_base and (page_title in data['title'] or match_base.group(0).strip() in page_title):
                    is_subpage_or_page = True

            if is_subpage_or_page:
                page_content_list.append({
                    'title': data['title'],
                    'content': data['revisions'][0],
                    'namespace': data['namespace']
                })

        combined_content = ""

        # 3-3. 수집된 모든 페이지 정렬 (숫자 기반 정렬 함수 사용) 및 조합
        if page_content_list:
            sorted_list = sort_wikisource_parts(page_content_list)

            for item in sorted_list:
                # 덤프에서 가져온 쪽 문서 텍스트에도 clean_text를 적용하여 불필요한 마크업을 제거
                cleaned_page_text = clean_text(item['content'])
                combined_content += cleaned_page_text.strip() + "\n\n"

        # 3-4. 내용이 비었으면 메인 페이지의 정리된 내용(목차 등)을 사용
        if not combined_content.strip():
            # '자유종'의 경우처럼 본문은 없지만 메타데이터는 남아있는 경우를 대비
            combined_content = clean_text(raw_content)

        # (생략: 최종 데이터 구조 생성 및 API 보강 로직은 이전과 동일)
        page_data = {
            'title': page_title, 'url': page_url, 'authors': metadata['authors'],
            'categories': metadata['categories'], 'content': combined_content.strip(),
            'raw_content': raw_content, 'year': metadata['year'], 'license': metadata['license'],
            'page_id': None, 'revision_id': None,
            'content_size': len(combined_content.strip()) # 정리된 본문의 길이 계산 및 추가
        }

        enhanced_data = enhance_with_api(page_data)
        final_data.append(enhanced_data)

    print(f"\n 총 {len(final_data)}개의 작품 데이터(덤프 기반) 추출 및 보강 완료.")

    return final_data

In [40]:
# 1. find_related_dump_pages 함수 정의
# 2. sort_wikisource_parts 함수 정의
# 3. process_csv_links_with_dump 함수 정의

# ----------------------------------------------------------------------
# 4. 실행
# ----------------------------------------------------------------------
dump_file = "kowikisource-20251001-pages-articles.xml.bz2"
csv_file = '한국근대소설_TEI_XML_작품목록.csv'

print("--- 덤프 기반 최종 파싱 시작 ---")
extracted_works = process_csv_links_with_dump(csv_file=csv_file, dump_file=dump_file)


# ----------------------------------------------------------------------
# 5. 결과 저장
# ----------------------------------------------------------------------
if extracted_works:
    results_df = pd.DataFrame(extracted_works)
    print("\n---  추출된 데이터 요약 (상위 5개) ---")
    print(results_df[['title', 'authors', 'year', 'year_from_wikidata', 'year_from_categories', 'license', 'url', 'content_size']].head())

    desired_format = 'xml'
    # output_data 함수 실행 (XML 변환 로직 포함)
    output_data(results_df, output_format=desired_format)
else:
    print("\n 최종 시도에서도 데이터 추출에 실패했습니다. 덤프 파일과 CSV 파일의 내용/경로를 확인해 주세요.")

--- 덤프 기반 최종 파싱 시작 ---

🔍 덤프 파일에서 42개 작품의 관련 페이지 찾기 시작...


덤프 페이지 탐색 중: 83815it [01:53, 736.29it/s] 


✅ 덤프에서 총 329개의 관련 페이지(본문, 하위 장, 쪽 문서) 수집 완료.


작품별 데이터 조합 및 보강: 100%|██████████| 42/42 [01:53<00:00,  2.70s/it]



 총 42개의 작품 데이터(덤프 기반) 추출 및 보강 완료.

---  추출된 데이터 요약 (상위 5개) ---
          title      authors  year  year_from_wikidata  year_from_categories  \
0           철세계  [이해조, 쥘 베른]  None                 NaN                   NaN   
1  어머니와 딸 (강경애)        [강경애]  1931                 NaN                1931.0   
2            무정        [이광수]  1918                 NaN                1918.0   
3           타락자        [현진건]  1922                 NaN                1922.0   
4            도정        [지하련]  1946                 NaN                1946.0   

      license                                                url  content_size  
0  PD-old-100  https://ko.wikisource.org/wiki/%EC%B2%A0%EC%84...         58894  
1   PD-old-50  https://ko.wikisource.org/wiki/%EC%96%B4%EB%A8...         83514  
2   PD-old-50  https://ko.wikisource.org/wiki/%EB%AC%B4%EC%A0%95        322845  
3   PD-old-50  https://ko.wikisource.org/wiki/%ED%83%80%EB%9D...         48806  
4   PD-old-50  https://ko.wikisource.org/wiki/%EB%